In [44]:
import pandas as pd
import re

In [45]:
df = pd.read_csv('crypto_tweets.csv')
df['Tweet'] = df['Tweet'].str.lower()
print(len(df))
df.head()

7012


,Datetime,Tweet,Username,Retweets,Followers
0,2021-10-28 06:07:54,what will happen to bitcoin once all 21 millio...,CryptoCryptoNe3,0.0,2797.0
1,2021-10-28 06:07:24,philly’s crossbeam — ‘tinder for business’ — l...,elobwana,0.0,1772.0
2,2021-10-28 06:07:24,the roadmap for banks adopting #bitcoin $btcus...,BTCstJournal,0.0,10867.0
3,2021-10-28 06:07:05,🚀🚀🚀 $icp long position(v1) 🚀🚀🚀\nentry price: 4...,HobaBot,0.0,19481.0
4,2021-10-28 06:07:04,📽️ krownscryptocave: 🛑live🛑 bitcoin a nasty tr...,CryptoTraderPro,0.0,29793.0


In [46]:
short_replace = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "ty": "thank you",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won’t": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you’re": "you are",
    "you've": "you have",
    "u.s.": "usa", "u.s.a" : "usa", 
    "btc": "bitcoin",
    "eth": "ethereum", "ethereumereum": "ethereum",
    "usdt": "tether",
    "ta": "technical analysis", 
    "ath": "all time high", "p2p": "peertopeer", 
    "bo": "breakout", "mo": "movement", 
    "w/": "with"
}

#replace shortcuts, words that have no meaning and also few symbols
df['Tweet'] = df['Tweet'].replace(short_replace, regex=True)
df['Tweet'] = df['Tweet'].str.replace("$", "#", regex = True)
df['Tweet'] = df['Tweet'].str.replace("’", "'", regex = True)

In [47]:
#Tweets with many hashtags, mentions and links with just a little text are usualy not very informative so I want to remove them

def meaningful_tweets(tweet):
    tweet = str(tweet)
    #get maximum of lenght and 1 to avoid getting 0 if there are no hashtags or links. To avoid division by 0 later. 1 instead of 0 is ok in this case...
    n_hashtags = max(len([i for i in tweet.split() if i.startswith('#')]), 1)
    n_links = max(len([i for i in tweet.split() if i.startswith('http')]), 1)
    n_mentions = max(len([i for i in tweet.split() if i.startswith('@')]), 1)
    n_words = max(len([i for i in tweet.split() if len(i) > 2]), 1) #number of words with more than 2 characters
    
    #get back only tweets where hashtags are maximum of 1/5 of total words and for every link there is at least 5 words
    if (n_words / n_hashtags) > 5 and (n_words / n_links) > 5 and (n_words / n_mentions) > 5:
        return tweet
    else:
        return ' ' #return empty string. Rows without values are dropped later so we get rid of this tweets...

In [48]:
def clean_tweet(tweet):
    tweet = str(tweet)
    tweet = ' '.join([w for w in tweet.split() if len(w) < 20]) #remove crypto addresses from strings
    tweet = ' '.join(re.sub("(#[A-Za-z0-9]+)", " ", tweet).split()) #remove hashtags
    tweet = ' '.join(re.sub("(&[A-Za-z0-9]+)", " ", tweet).split()) #remove hwords starting with &
    tweet = ' '.join(re.sub("(https?:?\/\/\S+)", " ", tweet).split()) #remove hyperlinks
    tweet = ' '.join(re.sub("(rt @[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) #remove RT and mentions
    tweet = ' '.join(re.sub("\S*\d\S*", " ", tweet).split()) #remove words containing numbers
    tweet = ' '.join(re.sub("([0-9])", " ", tweet).split()) #remove numbers
    
    #replace words that are spelled incorrectly
    tweet = ' '.join(['technical' if 'technic' in word else word for word in tweet.split()])
    tweet = ' '.join(['analysis' if 'analys' in word else word for word in tweet.split()])
    tweet = ' '.join(['ethereum' if 'thereu' in word else word for word in tweet.split()])
    tweet = ' '.join(['movement' if 'movement' in word else word for word in tweet.split()])
    tweet = ' '.join(['breakout' if 'breakout' in word else word for word in tweet.split()])
    
    return tweet

df['Tweet'] = df['Tweet'].apply(meaningful_tweets) #keep only meaningful tweets (remove with many hashtags and links and few real words)
df['Cleaned_Tweet'] = df['Tweet'].apply(clean_tweet) #clean tweets and save them in new column
df.drop_duplicates(subset='Cleaned_Tweet', keep="first", inplace = True) #drop duplicates from cleaned tweet column
df.dropna(subset=['Cleaned_Tweet']) #drop rows without any value
df = df[['Datetime', 'Tweet', 'Cleaned_Tweet']]
df = df[df['Cleaned_Tweet'].str.len() > 50] #keep only those tweets that are longer then 50 characters
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

print(len(df))
df.tail()

2334


,Datetime,Tweet,Cleaned_Tweet
2329,2021-10-28 06:13:55,#ren 15 minutes ☀️\n\nentry signal time: 27/10...,minutes entry signal time binance rentether di...
2330,2021-10-28 05:34:26,a friend of mine who cheated on me selling his...,a friend of mine who cheated on me selling his...
2331,2021-10-28 06:07:01,the post phemex launches turkey platform appea...,the post phemex launches turkey platform appea...
2332,2021-10-28 04:24:27,@thefinancebreakoutmb i have a lot there too 😉...,i have a lot there too was loving the on earni...
2333,2021-10-28 05:38:04,#shib it will go down faster than anything you...,it will go down faster than anything you have ...


In [49]:
df.to_csv('test_df.csv', index=None)